In [3]:
using GR
using Random
import Dates

In [4]:
#### specify path of Julia Code for 2D Models ####
path = "/home/jakob/Dokumente/Repositories/Modeling_Framework/2D_Models/"


##### include 2D Modelling Framework ####
include(string(path, "./2D_model_library.jl"));


########## OPERATIONAL ##########
##### how to chose the actual velocity? #####

op_model = "Constant"
#op_model = "Collision_Free_Speed"
#op_model = "RVO"
#op_model = "TTC_CostFunction"
#op_model = "SocialForce"
#op_model = "AV_Model"
#op_model = "Intrusion_Force"

include_op_model(op_model, path)

########## TACTICAL ##########
#### how to chose the preferred velocity? ###

#tact_model = "Constant"
tact_model = "Intrusion_Force"

include_tact_model(tact_model, path)


########## STRATEGIC ##########
### how to chose the desired velocity ###

strat_model = "Constant"
#strat_model = "Towards_Goal"

include_strat_model(strat_model, path)


##### which update scheme? #####

update = "Parallel_Update"
#update = "Step_Update"

include_update(update, path)


##### which order? i.e. Acceleration or Velocity based? ####
#order = "first"
order = "second"

include_order(order, path);

In [116]:
#### parameters of the agents ####
δt = 0.01
p = [1.6, 1.2, 3.0, 1.0, 0.3, 0.0, 0.0, 0.1, 0.05, 0.0, 0.0, 0.3, 0.6283185307179586]
p_desc = "v_max, v_des, T, T2, l, step_time, τ_A, τ_R, α, ζ_h, ζ_v, r_priv, ϕ"

### of the system ###
N = 2
system_size = (5.0, 5.0) #important for periodic boundaries
#system_size = (35.0, 35.0) #important for periodic boundaries

### of the simulation ###
sim_p = [0.0, 3.0, 0.04, δt, p[1]*p[2]] 
sim_p_desc = "t_relax, t_max, dt_save, dt, r"

### initialize geometry ###
geometrie = create_geometry_single_obstacle((37.5,31.5), 0.3)
#geometrie = Create_Geometry_Bottleneck(0.2, 0.8, 0.1, system_size)

### initialize crowd ###
menge = create_crowd(N, geometrie)

Init_Hom_Parameters!(p, menge)
Init_Hom_Vels!(menge, 1.0)

β, D, x_min, y_min, ϵ = π/2, sqrt(10), 0.0, 0.0, 0.001
Init_Two_Agents_α!(menge, β, D, x_min, y_min, ϵ)

#D, x_min, y_min, ϵ = 3.0, 0.5, 1.5, 0.2
#Init_Three_Agents!(menge, D, x_min, y_min, ϵ)

v2, v1 = 0.5, 1.5
T2, T1 = 1.0, 0.7
D, ϵ = 3.0, 0.0
x_min, y_min = 2.5, 2.5

#Init_Overtaking!(menge, D, x_min, y_min, ϵ, v1, v2, T1, T2);

### Sparse Bottleneck ###

#Init_Agent_Bottleneck!(1.9*pi, 4, menge, system_size)

#ϕ1, ϕ2, dist1, dist2 = 2.5*π/2, 3.5*π/2, 4.0, 4.0
#Init_2_Agents_Bottleneck!(ϕ1, ϕ2, dist1, dist2, menge, system_size)

#ϕ1, ϕ2, ϕ3, dist1, dist2, dist3 = 0.0, 3*π/2, 3.2*π/2, 4.0, 4.0, 3.5
#Init_3_Agents_Bottleneck!(ϕ1, ϕ2, ϕ3, dist1, dist2, dist3, menge, system_size)


#ϕ1, ϕ2, ϕ3, ϕ4, dist1, dist2, dist3, dist4 = 1.2*π, π, 3.2*π/2, 0.0, 4.0, 4.0, 3.8, 4.0
#Init_4_Agents_Bottleneck!(ϕ1, ϕ2, ϕ3, ϕ4, dist1, dist2, dist3, dist4, menge, system_size)

In [117]:
positions, headings, ttcs = Simulate!(menge, geometrie, sim_p[1], sim_p[2], sim_p[3], sim_p[4], sim_p[5], system_size);

In [118]:
GR.inline("mov")

j = 1

dt_save = sim_p[3]
geometry_x, geometry_y = rewrite_geometry(geometrie)
boundaries_x, boundaries_y = boundaries_rectangle_positions(system_size)

while j*sim_p[3] < sim_p[2]-sim_p[1]
    
        GR.clearws()

        GR.setwindow(-1, 6, -1, 6)
        GR.setviewport(0.0, 1, 0.0, 1)
    
        #plot_goal(goal1)
        #plot_goal(goal2)

        #plot_geometry(geometry_x, geometry_y, 3.6, 20)
        #plot_boundaries(boundaries_x, boundaries_y, 0.2, 999)        
        
        plot_agents(positions[j, :], headings[j, :], 3.6, 100, 1.2, 999, 0.1)
                
        model = "AV*IN-model"
        timer = string("time ", round(j*dt_save, digits = 1), "s")
        GR.text(0.8, 0.95, timer)
        GR.text(0.8, 0.9, model)
        
        GR.updatews()
    
        j = j+1
end

In [119]:
GR.show()

GR.HTML("<video autoplay controls><source type=\"video/mp4\" src=\"data:video/mp4;base64,AAAAFGZ0eXBxdCAgAAACAHF0ICAAAAAId2lkZQABRzJtZGF0AAACsQYF//+t3EXpvebZSLeWLNgg2SPu73gyNjQgLSBjb3JlIDE2MyAtIEguMjY0L01QRUctNCBBVkMgY29kZWMgLSBDb3B5bGVmdCAyMDAzLTIwMjEgLSBodHRwOi8vd3d3LnZpZGVvbGFuLm9yZy94MjY0Lmh0bWwgLSBvcHRpb25zOiBjYWJhYz0xIHJlZj0zIGRlYmxvY2s9MTowOjAgYW5hbHlzZT0weDM6MHgxMTMgbWU9aGV4IHN1Ym1lPTcgcHN5PTEgcHN5X3JkPTEuMDA6MC4wMCBtaXhlZF9yZWY9MSBtZV9yYW5nZT0xNiBjaHJvbWFfbWU9MSB0cmVsbGlzPTEgOHg4ZGN0PTEgY3FtPTAgZGVhZHpvbmU9MjEsMTEgZmFzdF9wc2tpcD0xIGNocm9tYV9xcF9vZmZzZXQ9LTIgdGhyZWFkcz0xMiBsb29rYWhlYWRfdGhyZWFkcz0yIHNsaWNlZF90aHJlYWRzPTAgbnI9MCBkZWNpbWF0ZT0xIGludGVybGFjZWQ9MCBibHVyYXlfY29tcGF0PTAgY29uc3RyYWluZWRfaW50cmE9MCBiZnJhbWVzPTMgYl9weXJhbWlkPTIgYl9hZGFwdD0xIGJfYmlhcz0wIGRpcmVjdD0xIHdlaWdodGI9MSBvcGVuX2dvcD0wIHdlaWdodHA9MiBrZXlpbnQ9MjUwIGtleWludF9taW49MjQgc2NlbmVjdXQ9NDAgaW50cmFfcmVmcmVzaD0wIHJjX2xvb2thaGVhZD00MCByYz1hYnIgbWJ0cmVlPTEgYml0cmF0ZT00MDAwIHJhdGV0b2w9MS4wIHFjb21wPTAuNjAgcXBtaW49MCBxcG1heD02OSBxcHN0ZXA9NCBpcF9yYXRpbz0xLjQwIGFxPTE6MS4wMACAAAAeRmWIhAAa//731LfMsu1Tb7YLqPeiniZfQ3UlAZuWxO06gAAAAwAAAwAAAwDpVIJjfO4A0Yy0AAAW43cqxW9GcwA/xIpoH0oiK0e6lVH4r43cNeuPzJijpQ8PVmBUHc/27v0J6vi2PmzIdBU6JN1XJZWIwH5bQyPdbzj01ocpcVhkHLxEJ6XUBK3ZBWNnsjjFzCXSpZg+aRVirLdKQim/Y9sjZw+RwU0ASkrwcOWTYw2/+e7XcVXrmhN5d0UbCxt8bejthpfjM0i8hv0q69aWnjEqyCmw++dH0+jdHkBbYCXZtqLgoZtywOYErIqkQTxTmoEkHItDxHDrTtlx+AsvM2p3faNFgngQnqqs5H+BWVIJvxaifcKiTmKEQ6xPWVTNhD6DgN9PnO/uYmnXq0zXYb3qctCsAvAsCu92xXBgCVk/R2cxJzkhvMg3QEYy6pI1/DRa/ZtsivvOMLN2PbwIjPmiux0O3uyGL/+R9m08x2omBwZ6NlVd2JIIyc2+i3V8DGPEmodA6Nb7BqKqa6WtsKvH10h0FXBpMHfROCptscMl0CvTYyWqtFlri+JE7tRz5HYcOqet/NK1hOeSjQgngbabd13hesxkuqvEN1zP8eR2o1Y+3lDzej7vpzv6dJgx5NBs/Sr+Xaf4w4uHKOdj3Mi063X2vAgeyktkC+iD0+p+1vL1RSvfVVFyPxVtAfDZ7EykKtPShQkQHuWhRrEnwiwUY+mzy70ZuV03VMppmCLlutR1Ynyg49dnUz2flvVyAcQzm+ztMFiz80L+b3evy8d1v5pRHhXkRU6XgV6+jKl9YJ1YKNE1VjqHg2ICX/sfmixkdC1N/45RBujNu8jm648MHt17M2yjKlLf4nHspYT2e4uXG/vrSZ9sdqkISAtyrWXTahlNpZ8XtTn6SE8hjsD6ZVyMeVbUYfvyRTr1WlNjcdPQnr5gHxCJ3/efYFoDJhAnI93iTzY5vPeO6PvuwOOWiTPFB5I661soY6dqgeu8jsC7FuLdNLi10+54+Wd0eLolMqBo61OJr0Xm+fwenUNrB09XTil/yz4m7i0wZHHWMW+XTjtnTWXgAFJBJqqv7zP9NF4bQKW/mz+A9ZEyWoclu/tY16fF6eLyTI1sVIwifP3Uif9SqQFwV0JcjvHytdASrWTtmbxx5geeja+u517v3UDbAzBH3pV2jezS5/z/tJ+G3IMJDECVQ2siQBLSQI4Rc6l/jCbE+M62tM5cBzMIzpPQx1q2CVF03zTfXOb4FR5G9So3qYBlMX42pvCcp5tPinPizitI9eQ/FbGkbvTIMovqpyh7QpieCCyQJWD9yZnamvhc4tVLp1u988HZSeNDRFdr07MFuy+MnQU0UBj4QWA8/fjRjsg5CG2NaTTImAHg9yy8lMc8L6isf1rSp55MIxzuNWYw3udcAdR1w9AA4ALsDIUJg1HhUObnHstljQ31uo9MYSGHKzPNSXesTl+oT/X96Ssybus0BvYgZwkg6iQmz0CPc4p37X41kell351qem32uL341fjgpXkN5vD33TZE+qDcbNBRdZcr/51wQMFvAMSeHU+of6CrWGrn1ifPm1wVu32HJV2YQ9JdxlxR7/iON3WPz2C/SyupxA6QFri6d6l6rvPtIezc7llxdtH7yyS7KqNMn3WmBKc6d9aAURvTRgAVUwcEnqO2l7kv8vx6ip5MgEZhbA1lUiCZdyXsvJcvbJl/dyXYm4pC77kbHvwQxpJv1WXoatTqLc9P+IF+uHTjs6nj3BgCfjAu2ZnA7jRjZ3Is1Z18FnxIeYfh1wx4KRTnriPTR2njCu8rX4LjjpGx+/rZ+MYlgYWEapUw1nFBuAjvu5ElzMz/Lqp3Z7DxmnRw5bMs/Qtp5FSEwgz8C909DJ1ge7Ublo2iszO3Di+6Tn2SGwhg/+7DtsVA/RmKh7Gif4Al2nDd9tkdAJgZbq1YcGtR051ld6npQy0EgEhEgr+TzldSUEeX3t5Kg8fLVbmZ9loH1SPmR6JZS0+1Bw5RGRUA0Ushevntz0UxKra4/ebMwdSFhh/bZajTSjUDRGqRicYybBG9KvJvfv84eT9IfeGxVQxSlbxgEh4r/aEC1lMJnIVmZONbjOMNCctwdTIrQl5W+Mw4Q+LNO6Tf5BOVVF4D1CdjZjKx+b8YQDtLBHduCOY4edkhRPf+cea+3BMU5sH1k7DYBBna/kf5sZMcEvHBMlm4b/cbNgjku0PBdwyT1h9tySrAfNgPqDm1VIylDCypY4Dd5QwjlYPh5OxI482avUKxjJCqJ7VMfwPj6AUNSmf6gFN/ebn83qYQ2vW6ud/oq5fGmIo9lPPKMScbBmEOoW59MH5L0yfXbbc8x6brMh4h8h0Vlb7mQHCBfaAypZx102hGmRGWWpw06naMOxT3yTF2aO3ykYL6o8S5sJRO9Og/drWv7gTijdfJWXR91b1sCv/oE5rKYScBh0gVbh+ggz6a1T+zBREt4EsQ6w4edkhRZrIP8G0KoJF+a+ID0U5Bve9Z4ORUzzeBqd6BAMPgEkGIGBpPKtMHAlxYuiCALJvJGLNEN0BvFef/QUfRJcwHyKcAuP723tGt5eaJaeyAET/cjz22Fll+G6CQhT2upaOJmS5PM6CjVNx8W0Vy3GySs+/dVqVEABrIb7iDZAE3ZCAcLrH3Uyd1n6Z+Crj2PPPetVQaKKJswXojVIxOMfCYIZuNnOzluLIl+kPtreATNWfImZLBFvyA+nGAHDRFTJOLILmmkRBrFawKahLyt8Zhwh8t3d7uEN5VoXFwCl6zRh0ywSejFwA7SwR3bgjmOHnZIUUmEPJ0+JfARXE3jKb8gjruSFM4e+mTHBLxwTJaha/1X0mI7F6ax9MMl8FbuKSqcxPMLs18bwumjkRHYYnBeqMG4oPh5OxJg/gevUKxjJCqJ7VMfwPj6AUbbY3

In [17]:
Path = "/home/jakob/Dokumente/Sim_Data/"
dir = "90_CollisionFreeSpeed"
header = Header(p, p_desc, sim_p, sim_p_desc, op_model, tact_model, order, update, N, system_size);

Save_Data!(Path, dir, header, positions)

Created Directory /90_CollisionFreeSpeed.


In [ ]:
samples = 15

for i in 1:samples
    
    Random.seed!()
    Init_Hom_Vels!(menge, 0.0)
    
    β, D, x_min, y_min, ϵ = pi, i/3, 2.5, 5.0, 0.00
    Init_Two_Agents_α!(menge, β, D, x_min, y_min, ϵ)

    
    positions, headings, ttcs = Simulate!(menge, geometrie, sim_p[1], sim_p[2], sim_p[3], sim_p[4], sim_p[5], system_size);
    Save_Data!(Path, dir, header, positions)
    
end

In [11]:
name = "Agents_Test"
path = "/home/jakob/Dokumente/Renne_Software/UMANS/examples/agents/"

Write_Agents_XML(menge, name, path)

298